# SE-Sync benchmarking

This notebook uses SE-Sync's Python bindings to permit easy benchmarking against a collection of large-scale SLAM benchmarks

In [ ]:
import sys
import numpy as np
import time

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display  # To get pretty tabular output w/ PANDAS
import pickle

import PySESync

### Test configuration

#### Test benchmarks

In [ ]:
# Location of folder containing the benchmarks
data_folder = "/home/dmrosen/SE-Sync/data/"

# Specific list of files to process
files = ["manhattan", "city10000", "MIT", "CSAIL", "intel", "sphere2500", "torus3D", "parking-garage", "rim"]
#files = ["sphere2500"]

#### SE-Sync configurations

In [ ]:
# Set list of SE-Sync configurations to process

# Common configurations
num_threads = 4
verbose = False

opts_list = [PySESync.SESyncOpts() for i in range(6)]

# Config 0: Simplified w/ chordal init
opts_list[0].formulation = PySESync.Formulation.Simplified
opts_list[0].initialization = PySESync.Initialization.Chordal
opts_list[0].num_threads = 4
opts_list[0].verbose = verbose

# Config 1: Simplified w/ random init
opts_list[1].formulation = PySESync.Formulation.Simplified
opts_list[1].initialization = PySESync.Initialization.Random
opts_list[1].num_threads = 4
opts_list[1].verbose = verbose

# Config 2: Explicit w/ chordal init
opts_list[2].formulation = PySESync.Formulation.Explicit
opts_list[2].initialization = PySESync.Initialization.Chordal
opts_list[2].num_threads = 4
opts_list[2].verbose = verbose

# Config 3: Explicit w/ random init
opts_list[3].formulation = PySESync.Formulation.Explicit
opts_list[3].initialization = PySESync.Initialization.Random
opts_list[3].num_threads = 4
opts_list[3].verbose = verbose

# Config 4: SOSync w/ chordal init
opts_list[4].formulation = PySESync.Formulation.SOSync
opts_list[4].initialization = PySESync.Initialization.Chordal
opts_list[4].num_threads = 4
opts_list[4].verbose = verbose

# Config 5: SOSync w/ random init
opts_list[5].formulation = PySESync.Formulation.SOSync
opts_list[5].initialization = PySESync.Initialization.Random
opts_list[5].num_threads = 4
opts_list[5].verbose = verbose


### Run tests!

In [ ]:
# Iterate over the list of benchmarks

data = []

start_time = time.time()
for k, f in enumerate(files):
    
    # Construct complete path to this file
    filename = data_folder + f + ".g2o"

    # Read measurements out of this file
    measurements, num_poses = PySESync.read_g2o_file(filename)
    
    # Get dimension of problem
    d = measurements[0].R.shape[0]
    
    # For each configuration in the test set 
    for c, opts in enumerate(opts_list):
        
        # Set initial relaxation rank to be the dimension of this test problem
        opts.r0 = d
        
        # Run SE-Sync with this configuration!
        result = PySESync.SESync(measurements, opts)
        
        # Record results for this test
        data.append({"Config": c, "Dataset" : f, "Status" : result.status.name,  \
                    "Fxhat" : result.Fxhat, "GradNorm" : result.gradnorm, \
                    "SubOptBound" : result.suboptimality_bound, \
                    "TotalTime" : result.total_computation_time, \
                    #"InitTime" : result.initialization_time, \
                    "OptTime" : sum(l[-1] for l in result.elapsed_optimization_times), \
                    "OptIters" : sum(len(l) for l in result.elapsed_optimization_times), \
                    "HessVecProds" : sum(map(sum, result.Hessian_vector_products)), \
                    "VerTime" : sum(result.verification_times), \
                    "VerIters" : sum(result.LOBPCG_iters), \
                    "FinalRank" : result.Yopt.shape[0]})
        

end_time = time.time()

elapsed_time = (end_time - start_time)

print("All tests finished.  Total computation time: %g seconds" % elapsed_time)

### Analysis

Create a pandas datafrom from the recorded results for further processing

In [ ]:
df = pd.DataFrame(data)
display(df)

Save this data to disk

In [ ]:
df.to_pickle("SLAM_benchmarking.pkl")

Read data from disk

In [ ]:
#df = pd.read_pickle("SESync_benchmarking.pkl")